In [1]:
using Pkg
Pkg.add("TestImages"); Pkg.add("Images");  Pkg.add("Colors")
using Knet, Random, IterTools, Base.Iterators, Images, Colors

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 % Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]


In [2]:
function mask!(a,pad)
    x,y = size(a)
    for i = 1:x
        for j = 1:y
            if a[i, j] == pad
                a[i, j] = 0
            end
        end
    end
    return a
end

mask! (generic function with 1 method)

In [3]:
wdict = Knet.load("dictionary.jld2", "dict")
UNK = 1
EOS = 2
w2i(x) = get(wdict, x, UNK)

w2i (generic function with 1 method)

In [4]:
dataset = Knet.load("data1.jld2", "dataset")
summary(dataset)

"5059-element Array{Tuple{Array{Float32,3},Array{Int64,1}},1}"

In [ ]:
y_truth = ones(Int32,length(dataset))
true_labels = ones(Int32,4183)
y_truth = y_truth[4184:end].+1
append!(true_labels, y_truth)
y_truth = copy(true_labels)
println(summary(y_truth))

rng = Random.seed!(0)
y_truth = shuffle(rng, y_truth)
rng = Random.seed!(0)
dataset = shuffle(rng, dataset)
println(summary(dataset))

In [6]:
VOCAB_SIZE = length(wdict)
BATCHSIZE = 40

40

In [7]:
mutable struct Conv 
    w 
    b 
    f_activation
    p_drop
    f_pool
end
(c::Conv)(x) = c.f_activation.(c.f_pool(conv4(c.w, dropout(x,c.p_drop)) .+ c.b))


Conv(w1::Int,w2::Int,cx::Int,cy::Int;f=relu, pdrop=0, f_pool=pool) = Conv(param(w1,w2,cx,cy), param0(1,1,cy,1), f, pdrop, f_pool)

Conv

In [8]:
struct Dense; w; b; f; p; end
(d::Dense)(x) = d.f.(d.w * mat(dropout(x,d.p)) .+ d.b) # mat reshapes 4-D tensor to 2-D matrix so we can use matmul
Dense(i::Int,o::Int,f=relu;pdrop=0) = Dense(param(o,i), param0(o), f, pdrop)

Dense

In [9]:
struct Projection; w end
(d::Projection)(x) = d.w * x
Projection(i::Int,o::Int) = Projection(param(o,i))

Projection

In [10]:
struct Embed; w; end
Embed(vocabsize::Int,embedsize::Int) = Embed(param(embedsize,vocabsize))
(e::Embed)(x) = e.w[:,x]

In [11]:
mutable struct frame_head
    conv1
    conv2
    conv3
    conv4
    fc
    output
end

function frame_head(w1,c1,w2,c2,w3,c3,w4,c4,hidden, outdims)
    conv1 = Conv(w1, w1, 1, c1)
    conv2 = Conv(w2, w2, c1, c2)
    conv3 = Conv(w3, w3, c2, c3)
    conv4 = Conv(w4, w4, c3, c4; f_pool = identity)
    fc = Dense(35840, hidden)
    output = Projection(hidden, outdims)
    frame_head(conv1, conv2, conv3, conv4, fc, output)
end
    

function (f::frame_head)(x)
    f.output(f.fc(f.conv4(f.conv3(f.conv2(f.conv1(x))))))
end

In [12]:
mutable struct sentence_head
    embed
    encoder
    fc
    output
end


function sentence_head(vocabsize::Int, embeddingsize::Int, hiddensize::Int)
    embed = Embed(vocabsize, embeddingsize)
    encoder = RNN(embeddingsize, hiddensize, rnnType = :lstm, h = 0)
    fc = Dense(8*hiddensize, 40)
    output = Projection(40,10)
    sentence_head(embed, encoder, fc, output)
end

sentence_head

In [13]:
function (s::sentence_head)(x)
    src_embed_tensor = s.embed(x)
    s.encoder.h = 0
    s.encoder.c = 0
    y_enc = s.encoder(src_embed_tensor)
    h, b, ty = size(y_enc)
    y_enc = reshape(permutedims(y_enc, (1,3,2)), (h*ty, b))
    s.output(s.fc(y_enc))
end


In [14]:
mutable struct bimodalEncoder
    fh
    sh
end


function bimodalEncoder(w1,c1,w2,c2,w3,c3,w4,c4,hidden, outdims, vocabsize, embeddingsize, hiddensize)
    fh = frame_head(w1,c1,w2,c2,w3,c3,w4,c4,hidden, outdims)
    sh = sentence_head(vocabsize, embeddingsize, hiddensize)
    bimodalEncoder(fh, sh)
end

function (b::bimodalEncoder)(image, label, y_truth)
    trans_nll(cosine_similarity(b, image_batch, seqbatch(labels)), y_truth)
end


function (b::bimodalEncoder)(x)

    images = []
    labels = []
   
    for i in 1:length(x)
        x1,x2 = x[i]
        
        push!(labels, x2)
        push!(images, x1)
    end
    
    image_batch = reshape(cat1d(images...), (210,160,1,length(x)))
    label_batch = permutedims(reshape(cat1d(labels...), (8, length(x))), (2,1))
    
    if (gpu()>=0)
        image_batch = KnetArray(image_batch)
    end
    
    cosine_similarity(b, image_batch, label_batch)
end



function(b::bimodalEncoder)(x,y)    
    trans_nll(b(x), y)
end

function (b::bimodalEncoder)(d::Knet.Data) 
    Knet.mean(b(x,y) for (x,y) in d)
end

In [15]:
function findmaxlength(sentences)
    maxsize = 0
    count = 0
    for sent in sentences
        count = count + 1
        if(length(sent) > maxsize)
            
            maxsize = length(sent)
        end
    end
    return maxsize
end

findmaxlength (generic function with 1 method)

In [16]:
function cosine_similarity(b::bimodalEncoder, frame_pairs, sentences)

    	
	sent_rep = b.sh(sentences)
	frame_rep = b.fh(frame_pairs)
	x = sum(sent_rep .* frame_rep, dims = 1)

	y1 = sqrt.(sum(sent_rep.^2,dims = 1))
	y2 = sqrt.(sum(frame_rep.^2,dims = 1))
	
	x ./ (y1 .* y2)
end

cosine_similarity (generic function with 1 method)

In [17]:
function convertsimilaritytoprobdist(cos_similarity)
    vcat(0.5.+((cos_similarity .* sqrt.(1 .- cos_similarity.^2))./π) + (asin.(cos_similarity)./π), 1 .- (0.5.+((cos_similarity .* sqrt.(1 .- cos_similarity.^2))./π) + (asin.(cos_similarity)./π)))
end

convertsimilaritytoprobdist (generic function with 1 method)

In [18]:
function trans_nll(cos_similarity, y_truth)
    nll(convertsimilaritytoprobdist(cos_similarity),y_truth)
end

trans_nll (generic function with 1 method)

In [17]:
sentt = []

open("instructions_unique.txt") do file
    for f in eachline(file)
    push!(sentt,f)    
    end
end


In [16]:
function reduce_sentence(input)
    
    pair_sentence = []
    
    for i in input
        for key in keys(wdict)
            if i == wdict[key]
                if key == "<eos>"
                    continue
                end
                if pair_sentence == []
                    pair_sentence = string(key," ")
                else
                    pair_sentence = string(pair_sentence, key," ")
                end
            end
        end
    end
    
    if (pair_sentence[1:21]==sentt[1])
        return sentt[1]
    elseif (pair_sentence[1:19]==sentt[2])
        return sentt[2]
    elseif (pair_sentence[1:11]==sentt[3])
        return sentt[3]
    elseif (pair_sentence[1:13]==sentt[4])
        return sentt[4]
    elseif (pair_sentence[1:13]==sentt[5])
        return sentt[5]
    elseif (pair_sentence[1:21]==sentt[6])
        return sentt[6]
    elseif (pair_sentence[1:28]==sentt[7])
        return sentt[7]
    elseif (pair_sentence[1:21]==sentt[8])
        return sentt[8]
    elseif (pair_sentence[1:28]==sentt[9])
        return sentt[9]
    elseif (pair_sentence[1:19]==sentt[10])
        return sentt[10]
    elseif (pair_sentence[1:31]==sentt[11])
        return sentt[11]
    elseif (pair_sentence[1:20]==sentt[12])
        return sentt[12]
    elseif (pair_sentence[1:32]==sentt[13])
        return sentt[13]
    elseif (pair_sentence[1:25]==sentt[14])
        return sentt[14]
    elseif (pair_sentence[1:18]==sentt[15])
        return sentt[15]
    elseif (pair_sentence[1:16]==sentt[16])
        return sentt[16]
    elseif (pair_sentence[1:11]==sentt[17])
        return sentt[17]
    elseif (pair_sentence[1:16]==sentt[18])
        return sentt[18]
    elseif (pair_sentence[1:12]==sentt[19])
        return sentt[19]
    elseif (pair_sentence[1:13]==sentt[20])
        return sentt[20]
    elseif (pair_sentence[1:9]==sentt[21])
        return sentt[21]
    end
end

reduce_sentence (generic function with 1 method)

In [20]:
d = dataset[1:13000]
y = y_truth[1:13000]
dtrn = minibatch(d, y, BATCHSIZE)
dtst = minibatch(dataset[13001:end], y_truth[13001:end], BATCHSIZE)
b = Knet.load("bimodal1.jld2","model")

summary(first(dtrn))

"Tuple{Array{Tuple{Array{Float32,3},Array{Int64,1}},1},Array{Int32,1}}"

In [1]:
# b = bimodalEncoder(8,64,5,64,4,128,3,128, 100, 10, length(wdict), 12, 10)
# Knet.save("bimodal1.jld2", "model", b)

UndefVarError: UndefVarError: wdict not defined

In [ ]:
# function accuracy(dataset, model)
#     total_count = 0
#     total_correct = 0
#     for batch in dataset
#         probdist = convertsimilaritytoprobdist(model(batch[1]))
#         preds = vec(map(x->x[1], argmax(probdist, dims=1)))
#         ground_truth = batch[2]
#         total_count += length(ground_truth)
#         total_correct += sum(ground_truth .== preds) 
#     end
#     return total_correct/total_count
# end

In [ ]:
# for i in 1:50
#     b = Knet.load("bimodal1.jld2", "model")
#     #println("Train error before training is: ", b(dtrn))
#     println("Test error before training is: ", b(dtst))
#     a = adam(b, take(cycle(dtrn),204))
#     progress!(a)
#     println("train accuracy is: ", accuracy(dtrn,b))
#     println("test accuracy is: ", accuracy(dtst, b))
#     Knet.save("bimodal1.jld2", "model", b)
# end

In [22]:
# # b = Knet.load("bimodal1.jld2", "model")
# println("Train error before training is: ", b(dtrn))
# println("Test error before training is: ", b(dtst))

Train error before training is: 0.7194562086190047
Test error before training is: 0.7324690840121669


In [ ]:
# a = adam(b, take(cycle(dtrn),250); gclip=0.5)
# progress!(a)

In [ ]:
# println("Train error after training is: ", b(dtrn))
# println("Test error after training is: ", b(dtst))

In [ ]:
# function accuracy(dataset, model)
#     total_count = 0
#     total_correct = 0
#     for batch in dataset
#         probdist = convertsimilaritytoprobdist(model(batch[1]))
#         preds = vec(map(x->x[1], argmax(probdist, dims=1)))
#         ground_truth = batch[2]
#         total_count += length(ground_truth)
#         total_correct += sum(ground_truth .== preds) 
#     end
#     return total_correct/total_count
# end

In [ ]:
# Knet.save("bimodal1.jld2", "model", b)

In [ ]:
#b = Knet.load("bimodalv6.jld2", "model")

In [ ]:
#println(firstbatch[2])

In [ ]:
#Knet.save("dtrnbackup.jld2", "model", dtrn)
#Knet.save("dtstbackup.jld2", "model", dtst)

In [ ]:
#dtrn = Knet.load("dtrnbackup.jld2", "model")
#dtst = Knet.load("dtstbackup.jld2", "model")
#summary(dtst)

In [ ]:
# println("train accuracy is: ", accuracy(dtrn,b))

In [ ]:
# println("test accuracy is: ", accuracy(dtst, b))

In [ ]:
#=
sample = rand(1:1:40)
f1,f2 = first(dtst)[1][sample][1]
both_frames = hcat(f1,f2)
img = permutedims(Float32.(both_frames./255), (3,1,2))

colorview(RGB, img)

=#

In [ ]:
#first(dtst)[1][sample][2]

In [ ]:
#first(dtst)[2][sample] 
#1 true
#2 false

In [ ]:
#argmax(convertsimilaritytoprobdist(b(first(dtst)[1]))[:,sample])